In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y #y와 x를 바꾼다
        dy = dout * self.x
        
        return dx, dy
    

In [5]:
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

#역전파
dprice = 1
dapple_price , dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num , dtax)

220.00000000000003
2.2 110.00000000000001 200


In [6]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x,y):
        out = x+y
        
        return out
    
    def backward(self,dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx,dy

In [10]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(all_price,tax)

#역전파
dprice = 1
dall_price ,dtax = mul_tax_layer.backward(dprice)
dapple_price,dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dall_price)
dapple,dapple_num = mul_apple_layer.backward(dall_price)

print(price)
print(dapple_num,dapple, dorange_num, dorange, dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


In [31]:
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    


In [43]:
x = np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)


[[ 1.  -0.5]
 [-2.   3. ]]


In [46]:
x = np.ndarray([[1.0,-0.5],[-2.0,3.0]])
print(x)
mask = (x <= 0)
print(mask)

TypeError: 'list' object cannot be interpreted as an integer

In [48]:
x = np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [50]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        
    def backward(self,dout):
        dx = dout * (1.0-self.out)*self.out
        
        return dx

In [52]:
x_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

In [53]:
x_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [54]:
x_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [55]:
dy = np.array([[1,2,3],[4,5,6]])
dy

array([[1, 2, 3],
       [4, 5, 6]])

In [56]:
dB= np.sum(dy, axis = 0)
dB


array([5, 7, 9])

In [57]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = none
        self.dW = none
        self.db = none
        
    def forward(self,x):
        self.x = x
        out = np.dot(x, self.W)+self.b
        
        return out
    
    def backward(self,dout):
        dx = np.dot(self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx
        

In [58]:
class SoftmasWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None # softmax의 출력
        self.y = None #정답레이블(원-핫 벡터)
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    
    def backward(self,dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t)/ batch_size
        return dx

In [63]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from ch05.layers import *
from ch05.gradient import numercial_gradient
from collections import OrderDict

class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size,weight_init_std=0.01):
        #가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)
        
        #계층생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \ 
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
                   
        self.lastLayer = SoftmaxWithLoss()
                   
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    #x: 입력데이터, t: 정답레이블
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
        
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t,axis =1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
                   
    #x는 입력데이터, t:정답테이블
    def numercial_gradient(self,x,t):
        loss_W = lambda W : self.loss(x,t)
        
        grads = {}
        grads['W1'] = numercial_gradient(loss_W,self.params['W1'])
        grads['b1'] = numercial_gradient(loss_W,self.params['b1'])
        grads['W2'] = numercial_gradient(loss_W,self.params['W2'])
        grads['b1'] = numercial_gradient(loss_W,self.params['b1'])
        return grads
                   
    def gradient(self,x,t):
        #순전파
        self.loss(x,t)
                   
        #역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
                   
        #결과저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b1'] = self.layers['Affine2'].db
        
        return grads

SyntaxError: unexpected character after line continuation character (<ipython-input-63-ab973e0613a3>, line 21)

In [69]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from ch04.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.1016754960655064e-07
b1:3.2132350723629385e-06
W2:6.282038168766709e-09
b2:1.3968668888669765e-07


In [ ]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from ch04.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)



0.1591 0.1539
0.90495 0.9071
0.9233666666666667 0.926
0.9361333333333334 0.9353
0.9446333333333333 0.9443
0.9502166666666667 0.9465
0.9559666666666666 0.9537
0.9604833333333334 0.9563
0.9628833333333333 0.9592
0.9656 0.962
0.9690666666666666 0.9644
0.9722166666666666 0.9652
